<a href="https://colab.research.google.com/github/riccardocappi/Text-Adversarial-Attack/blob/adversarial-training/adversarial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install textattack


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Imports

In [1]:
import textattack
import transformers
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.attack_recipes import BAEGarg2019

2024-05-20 14:52:16.245261: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 14:52:17.299547: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Helper methods

In [2]:
class FixedHuggingFaceDataset(HuggingFaceDataset):
    def __init__(self, name_or_dataset, subset=None, split="train", dataset_columns=None, label_map=None,
                 label_names=None, output_scale_factor=None, shuffle=False, seed=69, subset_size=None, offset=0):
        super().__init__(name_or_dataset=name_or_dataset, subset=subset, split=split, dataset_columns=dataset_columns,
                         label_map=label_map, label_names=label_names, output_scale_factor=output_scale_factor,
                         shuffle=shuffle)
        if shuffle:
            self._dataset = self._dataset.shuffle(seed=seed).flatten_indices()
        if subset_size is not None:
            self._dataset = self._dataset.skip(offset).take(subset_size)

# Loading model

In [3]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Loading dataset

In [4]:
subset_train = FixedHuggingFaceDataset("imdb", split="train", subset_size=1024, shuffle=True)
subset_eval = FixedHuggingFaceDataset("imdb", split="test", subset_size=64, shuffle=True)

textattack: Loading datasets dataset imdb, split train.
textattack: Loading datasets dataset imdb, split test.


# Adversarial training BAEGarg

In [5]:
attack = BAEGarg2019.build(model_wrapper)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


### Attack before model training

In [6]:
attack_args = textattack.AttackArgs(num_examples=-1, parallel=True, disable_stdout=True)
attacker = Attacker(attack, subset_eval, attack_args)
adv_exp_bae_bert_imdb = attacker.attack_dataset()

textattack: Running 1 worker(s) on 1 GPU(s).
2024-05-20 14:52:42.794637: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 14:52:43.430119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
textattack: Worklist size: 64
textattack: Worklist candidate size: 0
  0%|          | 0/64 [00:00<?, ?it/s]2024-05-20 14:53:30.785446: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1413 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
2024-05-20 14:53:31.279315: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system 

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 37 / 20 / 7 / 64: 100%|██████████| 64/64 [22:07<00:00, 20.74s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 37     |
| Number of failed attacks:     | 20     |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 89.06% |
| Accuracy under attack:        | 31.25% |
| Attack success rate:          | 64.91% |
| Average perturbed word %:     | 3.26%  |
| Average num. words per input: | 233.23 |
| Avg num queries:              | 331.86 |
+-------------------------------+--------+


### Define adversarial training hyperparams

In [7]:
training_args = textattack.TrainingArgs(
    num_epochs=5,
    num_clean_epochs=0,
    num_train_adv_examples=128,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,
    attack_epoch_interval=1,
    log_to_tb=True,
    parallel=True
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    subset_train,
    subset_eval,
    training_args
)

### Adversarial training

In [8]:
trainer.train()

textattack: Writing logs to ./outputs/2024-05-19-15-08-55-716471/train_log.txt.
textattack: Wrote original training args to ./outputs/2024-05-19-15-08-55-716471/training_args.json.
/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 1024
textattack:   Num epochs = 5
textattack:   Num clean epochs = 0
textattack:   Instantaneous batch size per device = 8
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 8
textattack:   Gradient accumulation steps = 1
textattack:   Total optimization steps = 720
textattack: ==========================================================
textattack: Epoch 1
text

Loss 0.21144: 100%|██████████| 144/144 [01:39<00:00,  1.45it/s]
textattack: Train accuracy: 93.58%
textattack: Eval accuracy: 92.19%
textattack: Best score found. Saved model to ./outputs/2024-05-19-15-08-55-716471/best_model/
textattack: ==========================================================
textattack: Epoch 2
textattack: Attacking model to generate new adversarial training set...
2024-05-19 16:22:51.747985: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 16:22:52.536151: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  0%|          | 0/128 [00:00<?, ?it/s]2024-05-19 16:23:04.222828: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/dev

Loss 0.16881: 100%|██████████| 144/144 [01:36<00:00,  1.49it/s]
textattack: Train accuracy: 95.23%
textattack: Eval accuracy: 90.62%
textattack: ==========================================================
textattack: Epoch 3
textattack: Attacking model to generate new adversarial training set...
2024-05-19 18:03:13.114522: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 18:03:17.941059: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  0%|          | 0/128 [00:00<?, ?it/s]2024-05-19 18:03:45.307145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3974 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:

Loss 0.14955: 100%|██████████| 144/144 [01:35<00:00,  1.51it/s]
textattack: Train accuracy: 95.83%
textattack: Eval accuracy: 87.50%
textattack: ==========================================================
textattack: Epoch 4
textattack: Attacking model to generate new adversarial training set...
2024-05-19 20:19:28.468860: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 20:19:29.180677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  0%|          | 0/128 [00:00<?, ?it/s]2024-05-19 20:19:39.972433: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3982 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:

Loss 0.13913: 100%|██████████| 144/144 [01:35<00:00,  1.51it/s]
textattack: Train accuracy: 95.31%
textattack: Eval accuracy: 93.75%
textattack: Best score found. Saved model to ./outputs/2024-05-19-15-08-55-716471/best_model/
textattack: ==========================================================
textattack: Epoch 5
textattack: Attacking model to generate new adversarial training set...
2024-05-19 23:53:23.594252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 23:53:24.301042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  0%|          | 0/128 [00:00<?, ?it/s]2024-05-19 23:53:35.275050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/dev

KeyboardInterrupt: 

# Check trained model accuracy

In [7]:
fine_tuned_model = transformers.AutoModelForSequenceClassification.from_pretrained("outputs/BAEGarg/best_model")
fine_tuned_tokenizer = transformers.AutoTokenizer.from_pretrained("outputs/BAEGarg/best_model")
model_wrapper_fine_tuned = textattack.models.wrappers.HuggingFaceModelWrapper(fine_tuned_model, fine_tuned_tokenizer)
fine_tuned_attack = BAEGarg2019.build(model_wrapper_fine_tuned)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [8]:
fine_tuned_attack_args = textattack.AttackArgs(num_examples=-1, parallel=True, disable_stdout=True)
fine_tuned_attacker = Attacker(fine_tuned_attack, subset_eval, fine_tuned_attack_args)
fine_tuned_eval = fine_tuned_attacker.attack_dataset()

textattack: Running 1 worker(s) on 1 GPU(s).
2024-05-20 15:14:58.177081: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 15:14:59.038900: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
textattack: Worklist size: 64
textattack: Worklist candidate size: 0
  0%|          | 0/64 [00:00<?, ?it/s]2024-05-20 15:15:10.533271: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1413 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
2024-05-20 15:15:11.025731: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system 

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 38 / 22 / 4 / 64: 100%|██████████| 64/64 [24:45<00:00, 23.21s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 38     |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 93.75% |
| Accuracy under attack:        | 34.38% |
| Attack success rate:          | 63.33% |
| Average perturbed word %:     | 4.26%  |
| Average num. words per input: | 233.23 |
| Avg num queries:              | 355.12 |
+-------------------------------+--------+
